In [1]:
import os
# 변경하고자 하는 디렉토리 경로
new_directory = r"C:\Users\user\Work\2024_가격예측모델"

# 현재 디렉토리를 새 경로로 변경
os.chdir(new_directory)
current_directory = os.getcwd()
current_directory

'C:\\Users\\user\\Work\\2024_가격예측모델'

In [2]:
current_directory

'C:\\Users\\user\\Work\\2024_가격예측모델'

In [3]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from tqdm.autonotebook import tqdm
import warnings
from src.utils.general import LogTime
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"
import matplotlib.pyplot as plt
import joblib
import random
from IPython.display import display, HTML
# %load_ext autoreload
# %autoreload 2
np.random.seed(42)
tqdm.pandas()

C:\Users\user\AppData\Local\Temp\ipykernel_54912\3185592865.py:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
# 데이터 디렉토리 설정
os.makedirs("imgs/chapter_6", exist_ok=True)
preprocessed = Path("raw_data/price_processesd_missing_date_additional")

## Reading the preprocessed and feature engineered files

In [5]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    display(train_df.head())
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

,room_id,room_name,hotel_id,hotel_name,ota_type,booking_date,day_of_week,dateName,high_demand,price_0,...,booking__Dayofweek_sin_1,booking__Dayofweek_sin_2,booking__Dayofweek_sin_3,booking__Dayofweek_sin_4,booking__Dayofweek_sin_5,booking__Dayofweek_cos_1,booking__Dayofweek_cos_2,booking__Dayofweek_cos_3,booking__Dayofweek_cos_4,booking__Dayofweek_cos_5
164200,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-01,Saturday,1월1일,Y,114000.0,...,-8.660254e-01,-8.660254e-01,2.388680e-15,8.660254e-01,8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5
164201,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-02,Sunday,N,N,45000.0,...,-2.449294e-16,-4.898587e-16,-7.347881e-16,-9.797175e-16,-4.777360e-15,1.0,1.0,1.0,1.0,1.0
164202,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-03,Monday,N,N,47500.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0
164203,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-04,Tuesday,N,N,47500.0,...,8.660254e-01,8.660254e-01,1.224647e-16,-8.660254e-01,-8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5
164204,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-05,Wednesday,N,N,47500.0,...,8.660254e-01,-8.660254e-01,-2.449294e-16,8.660254e-01,-8.660254e-01,-0.5,-0.5,1.0,-0.5,-0.5


In [6]:
train_df['price_0']=train_df['price_0'].astype('float64')

In [7]:
from src.transforms.target_transformations import AutoStationaryTransformer

In [8]:
#Takes a while. Snack Break!
transformer_pipelines = {}
for _id in tqdm(train_df["room_id"].unique()):
    #Initialize the AutoStationaryTransformer with a seasonality period of 1*7
    auto_stationary = AutoStationaryTransformer(seasonal_period=1*7)
    #Creating the timeseries with datetime index
    y = train_df.loc[train_df["room_id"]==_id, ["price_0","booking_date"]].set_index("booking_date")
    #Fitting and trainsforming the train
    y_stat = auto_stationary.fit_transform(y, freq="1D")
    # Setting the transformerd series back to the dataframe
    train_df.loc[train_df["room_id"]==_id, "price_0"] = y_stat.values
    #Saving the pipeline
    transformer_pipelines[_id] = auto_stationary

  0%|          | 0/278 [00:00<?, ?it/s]

In [9]:
train_df.head()

,room_id,room_name,hotel_id,hotel_name,ota_type,booking_date,day_of_week,dateName,high_demand,price_0,...,booking__Dayofweek_sin_1,booking__Dayofweek_sin_2,booking__Dayofweek_sin_3,booking__Dayofweek_sin_4,booking__Dayofweek_sin_5,booking__Dayofweek_cos_1,booking__Dayofweek_cos_2,booking__Dayofweek_cos_3,booking__Dayofweek_cos_4,booking__Dayofweek_cos_5
164200,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-01,Saturday,1월1일,Y,3.170998,...,-8.660254e-01,-8.660254e-01,2.388680e-15,8.660254e-01,8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5
164201,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-02,Sunday,N,N,3.164484,...,-2.449294e-16,-4.898587e-16,-7.347881e-16,-9.797175e-16,-4.777360e-15,1.0,1.0,1.0,1.0,1.0
164202,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-03,Monday,N,N,3.175203,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0
164203,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-04,Tuesday,N,N,3.177548,...,8.660254e-01,8.660254e-01,1.224647e-16,-8.660254e-01,-8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5
164204,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-05,Wednesday,N,N,3.176854,...,8.660254e-01,-8.660254e-01,-2.449294e-16,8.660254e-01,-8.660254e-01,-0.5,-0.5,1.0,-0.5,-0.5


## Saving the transformed file as well as the transformer pipelines

In [10]:
train_df = train_df.loc[:,["room_id","booking_date","price_0"]].set_index(["room_id","booking_date"])
train_df.rename(columns={"price_0":"price_0_auto_stat"}, inplace=True)


In [11]:
train_df.to_parquet(preprocessed/"selected_blocks_train_auto_stat_target.parquet")
joblib.dump(transformer_pipelines, preprocessed/"auto_transformer_pipelines_train.pkl")

['raw_data\\price_processesd_missing_date_additional\\auto_transformer_pipelines_train.pkl']

## train+val 저장하기

In [12]:
#Readin the missing value imputed and train test split data
try:
    train_df = pd.read_parquet(preprocessed/"selected_blocks_train_missing_imputed_feature_engg.parquet")
    val_df = pd.read_parquet(preprocessed/"selected_blocks_val_missing_imputed_feature_engg.parquet")
    train_df["type"] = "train"
    val_df["type"] = "val"
    train_df = pd.concat([train_df,val_df])
    del val_df
    display(train_df.head())
except FileNotFoundError:
    display(HTML("""
    <div class="alert alert-block alert-warning">
    <b>Warning!</b> File not found. Please make sure you have run 01-Feature Engineering.ipynb in Chapter06
    </div>
    """))

,room_id,room_name,hotel_id,hotel_name,ota_type,booking_date,day_of_week,dateName,high_demand,price_0,...,booking__Dayofweek_sin_2,booking__Dayofweek_sin_3,booking__Dayofweek_sin_4,booking__Dayofweek_sin_5,booking__Dayofweek_cos_1,booking__Dayofweek_cos_2,booking__Dayofweek_cos_3,booking__Dayofweek_cos_4,booking__Dayofweek_cos_5,type
164200,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-01,Saturday,1월1일,Y,114000.0,...,-8.660254e-01,2.388680e-15,8.660254e-01,8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5,train
164201,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-02,Sunday,N,N,45000.0,...,-4.898587e-16,-7.347881e-16,-9.797175e-16,-4.777360e-15,1.0,1.0,1.0,1.0,1.0,train
164202,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-03,Monday,N,N,47500.0,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0,1.0,train
164203,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-04,Tuesday,N,N,47500.0,...,8.660254e-01,1.224647e-16,-8.660254e-01,-8.660254e-01,0.5,-0.5,-1.0,-0.5,0.5,train
164204,186,스탠다드 트윈룸 도시 전망,115,베스트웨스턴해운대호텔,BOOKING,2022-01-05,Wednesday,N,N,47500.0,...,-8.660254e-01,-2.449294e-16,8.660254e-01,-8.660254e-01,-0.5,-0.5,1.0,-0.5,-0.5,train


In [13]:
train_df['price_0']=train_df['price_0'].astype('float64')

In [14]:
#Takes a while. Snack Break!
transformer_pipelines = {}
for _id in tqdm(train_df["room_id"].unique()):
    #Initialize the AutoStationaryTransformer with a seasonality period of 1*7
    auto_stationary = AutoStationaryTransformer(seasonal_period=1*7)
    #Creating the timeseries with datetime index
    y = train_df.loc[train_df["room_id"]==_id, ["price_0","booking_date"]].set_index("booking_date")
    #Fitting and trainsforming the train
    y_stat = auto_stationary.fit_transform(y, freq="1D")
    # Setting the transformerd series back to the dataframe
    train_df.loc[train_df["room_id"]==_id, "price_0"] = y_stat.values
    #Saving the pipeline
    transformer_pipelines[_id] = auto_stationary

  0%|          | 0/278 [00:00<?, ?it/s]

In [15]:
train_df = train_df.loc[:,["room_id","booking_date","price_0"]].set_index(["room_id","booking_date"])
train_df.rename(columns={"price_0":"price_0_auto_stat"}, inplace=True)
train_df.head()

price_0_auto_stat
room_id booking_date                   
186     2022-01-01             3.080700
        2022-01-02             3.074356
        2022-01-03             3.084667
        2022-01-04             3.086844
        2022-01-05             3.086163

In [16]:
train_df.to_parquet(preprocessed/"selected_blocks_train_val_auto_stat_target.parquet")
joblib.dump(transformer_pipelines, preprocessed/"auto_transformer_pipelines_train_val.pkl")

['raw_data\\price_processesd_missing_date_additional\\auto_transformer_pipelines_train_val.pkl']